<a href="https://colab.research.google.com/gist/shubham-ai/3fec76fa12dd418031a126a5ac080436/copy-of-baby_agi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BabyAGI User Guide

This notebook demonstrates how to implement [BabyAGI](https://github.com/yoheinakajima/babyagi/tree/main) by [Yohei Nakajima](https://twitter.com/yoheinakajima). BabyAGI is an AI agent that can generate and pretend to execute tasks based on a given objective, and you can find the origin notebook in [LangChain example](https://github.com/hwchase17/langchain/blob/master/docs/use_cases/autonomous_agents/baby_agi.ipynb).

This guide will help you understand the components to create your own recursive agents. And it will also show how to use GPTCache to cache the response.

Although BabyAGI uses specific vectorstores/model providers (Pinecone, OpenAI), one of the benefits of implementing it with LangChain is that you can easily swap those out for different options. In this implementation we use a Milvus vector datavase (because it runs locally and is free).

## Go into GPTCache

Please [install gptcache](https://gptcache.readthedocs.io/en/latest/index.html#) first, then we can initialize the cache.There are two ways to initialize the cache, the first is to use the map cache (exact match cache) and the second is to use the DataBse cache (similar search cache), it is more recommended to use the second one, but you have to install the related requirements.

> We will use [Milvus](https://milvus.io/) vector database to do similarity retrieval, so we need to install and start Milvus.


In [ ]:
! pip install -q gptcache langchain milvus tiktoken
!pip install grpcio==1.49.1
!pip install huggingface_hub > /dev/null

In [ ]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()


··········


In [ ]:
from milvus import default_server
default_server.start()



    __  _________ _   ____  ______
   /  |/  /  _/ /| | / / / / / __/
  / /|_/ // // /_| |/ / /_/ /\ \
 /_/  /_/___/____/___/\____/___/ {Lite}

 Welcome to use Milvus!

 Version:   v2.2.10-lite
 Process:   29129
 Started:   2023-06-28 09:03:45
 Config:    /root/.milvus.io/milvus-server/2.2.10/configs/milvus.yaml
 Logs:      /root/.milvus.io/milvus-server/2.2.10/logs

 Ctrl+C to exit ...


Before running the example, make sure the `OPENAI_API_KEY` environment variable is set by executing `echo $OPENAI_API_KEY`. If it is not already set, it can be set by using `export OPENAI_API_KEY=YOUR_API_KEY` on Unix/Linux/MacOS systems or `set OPENAI_API_KEY=YOUR_API_KEY` on Windows systems. And there is `get_content_func` for the cache settings:

> We can run `os.environ` to set the environment variable in colab.

In [ ]:
import os
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN



# get the content(only question) form the prompt to cache
def get_content_func(data, **_):
    return data.get("prompt").split("Question")[-1]

### 1. Init for exact match cache

In [ ]:
from gptcache import cache
cache.init(pre_embedding_func=get_content_func)
cache.set_openai_key()

### 2. Init for similar match cache

When initializing gptcahe, the following four parameters are configured:

- `pre_embedding_func`: pre-processing before extracting feature vectors, it will use the `get_content_func` method
- `embedding_func`: the method to extract the text feature vector
- `data_manager`: DataManager for cache management
- `similarity_evaluation`: the evaluation method after the cache hit

The `data_manager` is used to audio feature vector, response text in the example, it takes [Milvus](https://milvus.io/docs) (please make sure it is started), you can also configure other vector storage, refer to [VectorBase API](https://gptcache.readthedocs.io/en/latest/references/manager.html#module-gptcache.manager.vector_data).

In [ ]:
from gptcache import cache
from gptcache.embedding import Onnx
from gptcache.manager import CacheBase, VectorBase, get_data_manager
from gptcache.similarity_evaluation.distance import SearchDistanceEvaluation


onnx = Onnx()
cache_base = CacheBase('sqlite')
vector_base = VectorBase('milvus', host='127.0.0.1', port='19530', dimension=onnx.dimension)
data_manager = get_data_manager(cache_base, vector_base)
cache.init(
    pre_embedding_func=get_content_func,
    embedding_func=onnx.to_embeddings,
    data_manager=data_manager,
    similarity_evaluation=SearchDistanceEvaluation(),
    )
cache.set_openai_key()

INFO:gptcache:Attempting creation of Milvus index.
INFO:gptcache:Creation of Milvus index successful.


After initializing the cache, you can use the LangChain LLMs with `gptcache.adapter.langchain_models`. At this point **gptcache** will cache the answer, the only difference from the original example is to change `llm = OpenAI(temperature=0)` to `llm = LangChainLLMs(llm=OpenAI(temperature=0))`, which will be commented in the code block. And you can also set the `session` to set the session settings with `llm = LangChainLLMs(llm=OpenAI(temperature=0), session=session)`, more details refer to [session example](https://github.com/zilliztech/GPTCache/tree/main/examples#how-to-run-with-session).

Then you will find that it will be more fast when search the similar content, let's play with it.

## Install and Import Required Modules

In [ ]:
import os
from collections import deque
from typing import Dict, List, Optional, Any

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
from langchain.experimental import BabyAGI

from gptcache.adapter.langchain_models import LangChainLLMs
from gptcache.session import Session
session = Session(name="baby_agi") # set session for LangChainLLMs

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [ ]:
# from langchain.vectorstores import FAISS
from langchain.vectorstores import Milvus
from langchain.docstore import InMemoryDocstore

In [ ]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
embedding_size = 1536
vectorstore = Milvus(embeddings_model,
                     collection_name="baby_agi",
                     connection_args={"host": "127.0.0.1", "port": "19530"},
                    )

DEBUG:langchain.vectorstores.milvus:Using previous connection: e72f78c042f548baa87746ee1bad5f34


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [ ]:
OBJECTIVE = "Write a weather report for SF today"

NameError: ignored

In [ ]:
# llm = OpenAI(temperature=0)
# llm = LangChainLLMs(llm=OpenAI(temperature=0), session=session)

from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain

repo_id = "stabilityai/stablelm-tuned-alpha-3b"
 # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature": 0, "max_length": 64})


template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
# llm_chain = LLMChain(prompt=prompt, llm=llm)

# question = "Who won the FIFA World Cup in the year 1994? "

# print(llm_chain.run(question))
# # llm = LangChainLLMs(_llm, session=session)
# llm_chain = LLMChain(prompt=prompt, llm=llm)


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/stabilityai/stablelm-tuned-alpha-3b HTTP/1.1" 200 2214


In [ ]:
'0 aq2!-\P0O` 23 4"' Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm, vectorstore=vectorstore, verbose=verbose, max_iterations=max_iterations
)

In [ ]:
baby_agi({"objective": OBJECTIVE})

DEBUG:langchain.vectorstores.milvus:No existing collection to search.



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": ["You are an AI who performs one task based on the following objective: Write a weather report for SF today.Take into account these previously completed tasks: . Your task: Make a todo list. Response:"], "model": "text-davinci-003", "temperature": 0.0, "max_tokens": 256, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0, "n": 1, "logit_bias": {}}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/completions HTTP/1.1" 401 262
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=None request_id=cdbfa6ec6de7278d836f2c4b328082db response_code=401
INFO:openai:error_code=invalid_api_key error_message='Incorrect API key provided: YOUR_API_KEY. You can find your API key at https://platform.openai.com/account/api-keys.' error_param=None error_type=invalid_r

AuthenticationError: ignored